In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet18")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    for TL in range(2,-1,-1):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, test_loader1, test_loader3, test_loader5 = epg.getTrainDataset_val135(fold, sound[TL], num_class)

        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.220785140991211 Train_acc: 43.437500%
Test accuracy of the model: 22.500000%
Test accuracy of the model: 26.250000%
Test accuracy of the model: 26.250000%
Train Epoch: 2/30 Train_Loss: 0.7879501581192017 Train_acc: 75.937500%
Test accuracy of the model: 14.000000%
Test accuracy of the model: 15.000000%
Test accuracy of the model: 12.500000%
Train Epoch: 3/30 Train_Loss: 0.4307054877281189 Train_acc: 86.875000%
Test accuracy of the model: 11.500000%
Test accuracy of the model: 12.083333%
Test accuracy of the model: 11.250000%
Train Epoch: 4/30 Train_Loss: 0.33773842453956604 Train_acc: 85.937500%
Test accuracy of the model: 21.750000%
Test accuracy of the model: 27.916667%
Test accuracy of the model: 33.750000%
Train 

Test accuracy of the model: 76.666667%
Test accuracy of the model: 80.000000%
Train Epoch: 12/30 Train_Loss: 0.03608551621437073 Train_acc: 98.645836%
Test accuracy of the model: 66.750000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 80.000000%
Train Epoch: 13/30 Train_Loss: 0.05260490998625755 Train_acc: 98.854172%
Test accuracy of the model: 70.000000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 83.750000%
Train Epoch: 14/30 Train_Loss: 0.00876351073384285 Train_acc: 98.854172%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 77.916667%
Test accuracy of the model: 78.750000%
Train Epoch: 15/30 Train_Loss: 0.0024346744176000357 Train_acc: 98.854172%
Test accuracy of the model: 70.250000%
Test accuracy of the model: 78.750000%
Test accuracy of the model: 82.500000%
Train Epoch: 16/30 Train_Loss: 0.05600939691066742 Train_acc: 98.854172%
Test accuracy of the model: 69.500000%
Test accuracy of the model: 79.166667%
Test accu

Test accuracy of the model: 80.416667%
Test accuracy of the model: 78.750000%
Train Epoch: 24/30 Train_Loss: 0.06006012484431267 Train_acc: 94.812500%
Test accuracy of the model: 70.750000%
Test accuracy of the model: 77.916667%
Test accuracy of the model: 81.250000%
Train Epoch: 25/30 Train_Loss: 0.034338898956775665 Train_acc: 95.562500%
Test accuracy of the model: 69.750000%
Test accuracy of the model: 79.166667%
Test accuracy of the model: 77.500000%
Train Epoch: 26/30 Train_Loss: 0.1887185275554657 Train_acc: 95.500000%
Test accuracy of the model: 69.750000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 77.500000%
Train Epoch: 27/30 Train_Loss: 0.1099395677447319 Train_acc: 95.687500%
Test accuracy of the model: 70.500000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 78.750000%
Train Epoch: 28/30 Train_Loss: 0.04511117935180664 Train_acc: 95.562500%
Test accuracy of the model: 70.500000%
Test accuracy of the model: 80.000000%
Test accurac

Test accuracy of the model: 58.250000%
Test accuracy of the model: 65.416667%
Test accuracy of the model: 63.750000%
Train Epoch: 5/30 Train_Loss: 0.29258036613464355 Train_acc: 94.895836%
Test accuracy of the model: 72.750000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 85.000000%
Train Epoch: 6/30 Train_Loss: 0.14539633691310883 Train_acc: 95.937508%
Test accuracy of the model: 72.500000%
Test accuracy of the model: 88.333333%
Test accuracy of the model: 92.500000%
Train Epoch: 7/30 Train_Loss: 0.051864977926015854 Train_acc: 96.458336%
Test accuracy of the model: 70.500000%
Test accuracy of the model: 84.166667%
Test accuracy of the model: 87.500000%
Train Epoch: 8/30 Train_Loss: 0.07161835581064224 Train_acc: 96.354172%
Test accuracy of the model: 56.500000%
Test accuracy of the model: 66.250000%
Test accuracy of the model: 68.750000%
Train Epoch: 9/30 Train_Loss: 0.10432885587215424 Train_acc: 97.916672%
Test accuracy of the model: 68.250000%
Test accuracy o

Test accuracy of the model: 81.250000%
Test accuracy of the model: 90.833333%
Test accuracy of the model: 92.500000%
Train Epoch: 17/30 Train_Loss: 0.16455389559268951 Train_acc: 94.000000%
Test accuracy of the model: 82.250000%
Test accuracy of the model: 90.416667%
Test accuracy of the model: 92.500000%
Train Epoch: 18/30 Train_Loss: 0.0667157992720604 Train_acc: 93.625000%
Test accuracy of the model: 80.750000%
Test accuracy of the model: 89.166667%
Test accuracy of the model: 91.250000%
Train Epoch: 19/30 Train_Loss: 0.18303000926971436 Train_acc: 94.250000%
Test accuracy of the model: 81.000000%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 91.250000%
Train Epoch: 20/30 Train_Loss: 0.2919695973396301 Train_acc: 93.812500%
Test accuracy of the model: 81.750000%
Test accuracy of the model: 90.833333%
Test accuracy of the model: 92.500000%
Train Epoch: 21/30 Train_Loss: 0.11630894243717194 Train_acc: 94.437500%
Test accuracy of the model: 81.000000%
Test accuracy

...

Train Epoch: 9/30 Train_Loss: 0.15156850218772888 Train_acc: 91.437500%
Test accuracy of the model: 73.250000%
Test accuracy of the model: 79.583333%
Test accuracy of the model: 83.750000%
Train Epoch: 10/30 Train_Loss: 0.20606252551078796 Train_acc: 93.437500%
Test accuracy of the model: 73.000000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 83.750000%
Train Epoch: 11/30 Train_Loss: 0.09061052650213242 Train_acc: 93.687500%
Test accuracy of the model: 75.250000%
Test accuracy of the model: 79.583333%
Test accuracy of the model: 81.250000%
Train Epoch: 12/30 Train_Loss: 0.21920520067214966 Train_acc: 94.250000%
Test accuracy of the model: 75.500000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 85.000000%
Train Epoch: 13/30 Train_Loss: 0.12814468145370483 Train_acc: 94.625000%
Test accuracy of the model: 77.250000%
Test accuracy of the model: 83.333333%
Test accuracy of the model: 85.000000%
Train Epoch: 14/30 Train_Loss: 0.11898108571767807 

Train Epoch: 21/30 Train_Loss: 0.010900183580815792 Train_acc: 100.000000%
Test accuracy of the model: 66.000000%
Test accuracy of the model: 81.666667%
Test accuracy of the model: 88.750000%
Train Epoch: 22/30 Train_Loss: 0.017095141112804413 Train_acc: 100.000000%
Test accuracy of the model: 67.250000%
Test accuracy of the model: 82.916667%
Test accuracy of the model: 90.000000%
Train Epoch: 23/30 Train_Loss: 0.008828867226839066 Train_acc: 100.000000%
Test accuracy of the model: 68.750000%
Test accuracy of the model: 80.833333%
Test accuracy of the model: 87.500000%
Train Epoch: 24/30 Train_Loss: 0.010079544968903065 Train_acc: 100.000000%
Test accuracy of the model: 68.750000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 88.750000%
Train Epoch: 25/30 Train_Loss: 0.009300285018980503 Train_acc: 100.000000%
Test accuracy of the model: 68.250000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 88.750000%
Train Epoch: 26/30 Train_Loss: 0.0097090

Test accuracy of the model: 82.500000%
Test accuracy of the model: 83.750000%
Train Epoch: 2/30 Train_Loss: 0.19474422931671143 Train_acc: 87.812500%
Test accuracy of the model: 70.000000%
Test accuracy of the model: 76.250000%
Test accuracy of the model: 76.250000%
Train Epoch: 3/30 Train_Loss: 0.2793537676334381 Train_acc: 89.812500%
Test accuracy of the model: 69.500000%
Test accuracy of the model: 72.083333%
Test accuracy of the model: 73.750000%
Train Epoch: 4/30 Train_Loss: 0.20828692615032196 Train_acc: 91.562500%
Test accuracy of the model: 73.250000%
Test accuracy of the model: 79.583333%
Test accuracy of the model: 80.000000%
Train Epoch: 5/30 Train_Loss: 0.20308487117290497 Train_acc: 91.937500%
Test accuracy of the model: 74.000000%
Test accuracy of the model: 83.750000%
Test accuracy of the model: 87.500000%
Train Epoch: 6/30 Train_Loss: 0.21360865235328674 Train_acc: 92.125000%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 74.583333%
Test accuracy of 

Test accuracy of the model: 71.250000%
Test accuracy of the model: 78.750000%
Train Epoch: 14/30 Train_Loss: 0.04304880276322365 Train_acc: 99.687500%
Test accuracy of the model: 65.750000%
Test accuracy of the model: 75.416667%
Test accuracy of the model: 83.750000%
Train Epoch: 15/30 Train_Loss: 0.06980472803115845 Train_acc: 99.375000%
Test accuracy of the model: 67.250000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 83.750000%
Train Epoch: 16/30 Train_Loss: 0.04484499245882034 Train_acc: 100.000000%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 80.833333%
Test accuracy of the model: 88.750000%
Train Epoch: 17/30 Train_Loss: 0.015319290570914745 Train_acc: 100.000000%
Test accuracy of the model: 66.250000%
Test accuracy of the model: 78.750000%
Test accuracy of the model: 86.250000%
Train Epoch: 18/30 Train_Loss: 0.020370399579405785 Train_acc: 99.687500%
Test accuracy of the model: 66.750000%
Test accuracy of the model: 77.083333%
Test ac

Test accuracy of the model: 73.250000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 87.500000%
Train Epoch: 26/30 Train_Loss: 0.015017678029835224 Train_acc: 98.541672%
Test accuracy of the model: 73.250000%
Test accuracy of the model: 84.583333%
Test accuracy of the model: 87.500000%
Train Epoch: 27/30 Train_Loss: 0.10358507931232452 Train_acc: 98.437508%
Test accuracy of the model: 74.250000%
Test accuracy of the model: 85.416667%
Test accuracy of the model: 86.250000%
Train Epoch: 28/30 Train_Loss: 0.02451115846633911 Train_acc: 98.541672%
Test accuracy of the model: 76.250000%
Test accuracy of the model: 87.916667%
Test accuracy of the model: 88.750000%
Train Epoch: 29/30 Train_Loss: 0.013433185406029224 Train_acc: 98.541672%
Test accuracy of the model: 74.500000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 87.500000%
Train Epoch: 30/30 Train_Loss: 0.03670269995927811 Train_acc: 98.437508%
Test accuracy of the model: 75.750000%
Test accu

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()